<a href="https://colab.research.google.com/github/markwang0/GeoFlood/blob/WIP-pygeoflood-class/examples/geoflood_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install in-development branch of GeoFlood

In [1]:
%%capture

inColab = 'google.colab' in str(get_ipython())
if inColab:
    !pip install -e git+https://github.com/markwang0/geoflood@WIP-pygeoflood-class
else:
    !pip install -e .


### Run GeoFlood operations

In [5]:
import os
from pathlib import Path
from pygeoflood import PyGeoFlood

my_dem_path = Path("data", "OC1mTest.tif")
# download DEM if it doesn't exist
if not my_dem_path.is_file():
    !curl https://utexas.box.com/shared/static/id0in7w8yn38o59sexa50aqunsr7mzk0.tif -Lso data/OC1mTest.tif
geoflood_onion = PyGeoFlood(dem_path=my_dem_path)
print(geoflood_onion)

PyGeoFlood(
    dem_path='data/OC1mTest.tif'
    project_dir='data'
    dem_smoothing_quantile=0.9
)


In [6]:
# Perona-Malik filtered DEM saved as
# [DEM_name]_PM_filtered.tif
# in same directory as the original DEM
geoflood_onion.nonlinear_filter()

Computing slope of raw DTM
DEM slope array shape: (3378, 4750)
Computing lambda = q-q-based nonlinear filtering threshold
DEM smoothing Quantile: 0.9
Edge Threshold Value: 0.34356024861335754
Time taken to complete nonlinear filtering: 12.0 seconds


In [7]:
print(geoflood_onion)

PyGeoFlood(
    dem_path='data/OC1mTest.tif'
    project_dir='data'
    dem_smoothing_quantile=0.9
    filtered_dem_path='data/OC1mTest_PM_filtered.tif'
)


### pysheds replacement for grass_py3.py

In [3]:
from pysheds.grid import Grid
import numpy as np
import os

# compare with demo_geoflood dir
my_pm_filt_path = os.path.expanduser(
    "~/Documents/pygeoflood_demo/OC1mTest_PM_filtered.tif"
)
grid = Grid.from_raster(my_pm_filt_path)
dem = grid.read_raster(my_pm_filt_path)
grass_dirmap = (2, 1, 8, 7, 6, 5, 4, 3)
fdir = grid.flowdir(dem, dirmap=grass_dirmap, routing="mfd")
acc = grid.accumulation(fdir, dirmap=grass_dirmap, routing="mfd")

In [20]:
print(np.nanmin(acc), np.nanmax(acc))
print(np.nanmin(fdir), np.nanmax(fdir))

1.0 399245.8507925526
0.0 1.0


In [28]:
np.nanmax(fdir[1, :, :])

MultiRaster(1.)

In [46]:
np.nanmin(np.array(acc))

1.0

In [58]:
fdir_inf = grid.flowdir(dem, dirmap=grass_dirmap, routing="dinf")
acc_inf = grid.accumulation(fdir_inf, dirmap=grass_dirmap, routing="dinf")

In [60]:
np.sum(np.array(fdir_inf) < 0)

76041

In [61]:
np.sum(np.array(fdir) < 0)

0

In [62]:
np.sum(np.array(acc) < 0)

0

In [12]:
grid.to_raster(
    acc,
    "/Users/markwang/Documents/pygeoflood_demo/acc.tif",
    blockxsize=16,
    blockysize=16,
)